In [5]:
from constants import DEV_BUCKET, STAGE_BUCKET, PROD_BUCKET, FOLDER_NAME
import tempfile
from dvc.api import DVCFileSystem
import s3fs
import yaml
import datetime

In [6]:
repo = 'git@github.com:ContextLogic/multitask-llm-rnd.git'
path = '/modelling/notebooks/convert_pl_to_hf_ckpt/query_classify_v3_mini/hf_ckpt'

In [7]:
MODEL_NAME = "query-classify/v3"

In [8]:
s3_file = s3fs.S3FileSystem()

In [10]:
s3_file.ls(f"{DEV_BUCKET}/xyz/{MODEL_NAME}")

[]

In [4]:
with tempfile.TemporaryDirectory() as tmpdirname:
    print(tmpdirname)
    fs = DVCFileSystem(repo, subrepos=True)
    fs.get(path, tmpdirname, recursive=True)
    s3_file = s3fs.S3FileSystem()
    local_path = tmpdirname
    
    def rm_and_save(s3_path):
        if len(s3_file.ls(s3_path)) > 0:
            try:
                print(f'rm -rf {s3_path}')
                s3_file.rm(s3_path, recursive=True)
            except Exception as e:
                print(f'rm failed due to {e}')
        print(f'save to {s3_path}')
        s3_file.put(local_path, s3_path, recursive=True) 

    rm_and_save(f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}")
    rm_and_save(f"{STAGE_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}")
    rm_and_save(f"{PROD_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}")
    

/tmp/tmpbk0yai24
save to structured-data-dev/coeus-gpu-multitask-ml/query-classify/v3
save to structured-data-stage/coeus-gpu-multitask-ml/query-classify/v3
save to structured-data-prod/coeus-gpu-multitask-ml/query-classify/v3


In [4]:
yaml_dict = yaml.safe_load(open('models.yaml', 'r'))

In [5]:
if yaml_dict is None:
    yaml_dict = {}

In [6]:
assert MODEL_NAME.count('/') == 1

In [7]:
model_class, model_version = MODEL_NAME.split('/')

In [9]:
if model_class not in yaml_dict:
    yaml_dict[model_class] ={}
assert model_version not in yaml_dict[model_class]
yaml_dict[model_class][model_version] = {
    "dvc": {
        "repo": "git@github.com:ContextLogic/multitask-llm-rnd.git", 
        "path": "/modelling/notebooks/convert_pl_to_hf_ckpt/query_classify_v3_mini/hf_ckpt", 
        "rev": "1c1262ab17874ff326866c0843d135e083da3af2"
    },
    "s3": {
        "dev": f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}",
        "stage": f"{STAGE_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}",
        "prod": f"{PROD_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}"
    },
    "metadata": {
        "library_type": "huggingface-pipeline",
        "model_type": "text-classification-multilabel",
        "yaml_update_time": str(datetime.datetime.now()),
        "taxonomy_version": "v1.2.1",
        "max_batch_size": 100,
        "num_top_preds": 3
    }
}

In [10]:
with open('models.yaml', 'w') as f:
    f.write(yaml.dump(yaml_dict))

In [4]:
f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}"

'structured-data-dev/coeus-gpu-multitask-ml/query-classify/v3'

In [7]:
help(fs.get)

Help on function _get in module fsspec.asyn:

_get(rpath, lpath, recursive=False, callback=<fsspec.callbacks.NoOpCallback object at 0x7f8d0bb65fa0>, **kwargs)
    Copy file(s) to local.
    
    Copies a specific file or tree of files (if recursive=True). If lpath
    ends with a "/", it will be assumed to be a directory, and target files
    will go within. Can submit a list of paths, which may be glob-patterns
    and will be expanded.
    
    The get_file method will be called concurrently on a batch of files. The
    batch_size option can configure the amount of futures that can be executed
    at the same time. If it is -1, then all the files will be uploaded concurrently.
    The default can be set for this instance by passing "batch_size" in the
    constructor, or for all instances by setting the "gather_batch_size" key
    in ``fsspec.config.conf``, falling back to 1/8th of the system limit .



In [5]:
import s3fs
fs = s3fs.S3FileSystem()

In [8]:
fs.get(f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}", "tmp", recursive=True)

[None, None, None, None, None, None, None, None, None]